这个 Notebook 的用途是展示如何结合使用 Ollama 和 OpenAI，并演示 OpenAI 模型微调（Fine-tuning）的基本流程。它会引导您了解如何利用这些工具进行模型微调。

### 安装 Ollama
---

- Ollama 需要 pciutils 来检测 GPU 类型。
- 在运行时实例中安装 Ollama 将由以下命令处理：`curl -fsSL https://ollama.com/install.sh | sh`

In [1]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Get:4 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,248 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InReleas

In [2]:
!ollama --version

要使用 Ollama，它需要作为后台服务与脚本并行运行。由于 Jupyter Notebook 设计为顺序执行代码块，这使得同时运行两个代码块变得困难。作为变通方案，我们将使用 Python 的 subprocess 创建服务，确保其不会阻塞任何单元格的执行。

通过命令 `ollama serve` 可启动服务。

`time.sleep(5)` 添加了延迟，确保 Ollama 服务启动完成后再下载模型。

In [3]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

### 拉取模型
---

使用 `ollama pull qwen3:4b` 下载 LLM 模型。

其他模型请访问 [https://ollama.com/library](https://ollama.com/library)

In [4]:
!ollama pull qwen3:4b

### OpenAI 模型微调（Fine-tuning）

In [5]:
!pip install langchain[openai]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 7.8 MB/s eta 0:00:00


准备训练数据（JSONL）

In [3]:
import json

data = [
    {
        "messages": [
            {"role": "system", "content": "你是一个电力行业数据分析专家"},
            {"role": "user", "content": "今年江苏售电量情况如何？"},
            {"role": "assistant", "content": "今年江苏整体售电量同比小幅增长，工业用电恢复明显。"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "你是一个电力行业数据分析专家"},
            {"role": "user", "content": "分析一下售电到户均价"},
            {"role": "assistant", "content": "售电到户均价受市场交易电价和政策调控双重影响，整体趋稳。"}
        ]
    }
]

with open("train.jsonl", "w", encoding="utf-8") as f:
    for item in data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

由于是利用 Ollama 部署的模型，所以无法使用 OpenAI SDK 调用 Ollama 微调模型

原因是 Ollama 只模拟了 OpenAI Chat API 协议，即 `Chat/Completion` 与部分 `Embeddings`

下面是 OpenAI SDK 微调模型的示例，示例结果会报错 `NotFoundError: 404 page not found`

In [17]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

# 上传训练数据
file = client.files.create(file=open("train.jsonl", "rb"), purpose="fine-tune")

# 启动微调任务
job = client.fine_tuning.jobs.create(training_file=file.id, model="qwen3:4b")

print(job)

NotFoundError: 404 page not found

OpenAI 检查微调任务的状态

In [ ]:
client.fine_tuning.jobs.retrieve(job.id)

#### 什么是 LoRA 和 QLoRA
---

简单来说，LoRA 和 QLoRA 是目前大语言模型（LLM）微调领域最受欢迎的技术。它们的核心目标一致：让你用极低的算力成本，去训练原本需要昂贵显卡才能跑动的大模型。

QLoRA 是华盛顿大学研究者提出的对 LoRA 的升级版，QLoRA 的核心是引入量化技术压缩了基础模型的内存占用对 LoRA 改进，使得在单张消费级显卡上能微调极大的模型。

#### 什么是 Unsloth
---

Unsloth 是目前开源社区中最受欢迎的 LLM（大语言模型）微调训练框架，Unsloth 专门针对 LoRA/QLoRA 进行了底层优化，其性能表现非常惊人。

#### 使用 Unsloth 进行 LoRA 微调
---

流程：
- 使用 Unsloth 框架进行 LoRA 微调。
- 将微调后的 LoRA 权重与原模型合并（Merge）。
- 导出为 GGUF 格式。
- 导入 Ollama 进行部署。

In [2]:
!pip install unsloth transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 2.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.1/381.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 137.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/8

In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments

max_seq_length = 2048 # 可以增加更长的推理轨迹
lora_rank = 16 # rank 值越大越聪明，但速度也越缓慢

# 1. 配置模型与硬件优化
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Base",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # 使用 4bit 量化以节省显存
    max_lora_rank = lora_rank,
)

# 2. 设置 LoRA 参数
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # 选择任何大于0的数字！建议8、16、32、64、128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = lora_rank*2, # *2倍速训练
    use_gradient_checkpointing = "unsloth", # 可以减少内存使用
    random_state = 3407,
)

# 3. 准备数据集 (以简单的问答为例)
dataset = load_dataset("jsonl", data_files="train.jsonl", split="train")

# 4. 设置训练参数
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = TrainingArguments(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1,
        warmup_steps = 5,
        num_train_epochs = 2,
        learning_rate = 2e-4,
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none",
        output_dir = "outputs",
    ),
)

# 5. 开始训练
trainer.train()

# 6. 关键步骤：导出为 Ollama 兼容的 GGUF 格式
# 这步会自动合并权重并量化
model.save_pretrained_gguf("my_model_gguf", tokenizer, quantization_method = "q8_0")

#### 将模型导入 Ollama

训练完成后，会得到一个名为 `my_model_gguf` 的文件夹，里面包含一个 `.gguf` 文件。

编写 Modelfile：

In [ ]:
modelfile = '''
FROM ./my_model_gguf/unsloth.Q8_0.gguf

# 设置模板（根据你训练时的 Prompt 格式调整）
TEMPLATE """{{ .System }}
### Instruction:
{{ .Prompt }}

### Response:
{{ .Response }}"""

# 设置参数
PARAMETER stop "### Response:"
'''

with open("Modelfile", "w", encoding="utf-8") as f:
    f.write(modelfile)

In [ ]:
!ollama create qwen3-4b-finetuned -f Modelfile

#### 使用微调模型调用问答

In [6]:
from IPython.display import Markdown

from langchain.chat_models.base import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser

model_name = "qwen3:4b"
# model_name = "qwen3-4b-finetuned"

template = """问题：{query}
答案：一句话回答"""

prompt = ChatPromptTemplate.from_template(template)

model = init_chat_model(
    model=model_name,
    model_provider="openai",
    base_url="http://localhost:11434/v1",
    api_key="ollama",
)

chain = prompt | model | StrOutputParser()

display(Markdown(chain.invoke({"query": "分析一下售电到户均价"})))

售电到户均价指电力企业向终端用户销售电能的平均单位价格，由国家政策指导、供电成本及市场供需三方面共同决定，且通常根据用电类型（如居民、工商业）实施阶梯式电价分档计价。